In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

In [14]:
%config InlineBackend.figure_format = "retina"
sns.set()

In [15]:
df = pd.read_csv("data_character_meaning.csv", index_col=0)

/Users/aureliennioche/.pyenv/versions/3.9.1/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
df

,user,domain,condition,item,item_character,item_meaning,success,teacher_md,learner_md,psy_md,session,is_eval,ts_display,ts_reply,n_session_done
0,goldfish@active.fi,active.fi,ThresholdCondition,1506,嬢,miss,True,leitner,NaN,grid,1900,False,2020-09-30 17:51:37.666000+00:00,2020-09-30 17:51:41.042000+00:00,14
1,goldfish@active.fi,active.fi,ThresholdCondition,190,姉,older sister,True,leitner,NaN,grid,1900,False,2020-09-30 17:51:42.081000+00:00,2020-09-30 17:51:43.910000+00:00,14
2,goldfish@active.fi,active.fi,ThresholdCondition,1000,婚,marriage,True,leitner,NaN,grid,1900,False,2020-09-30 17:51:44.980000+00:00,2020-09-30 17:51:46.413000+00:00,14
3,goldfish@active.fi,active.fi,ThresholdCondition,1506,嬢,miss,True,leitner,NaN,grid,1900,False,2020-09-30 17:51:47.476000+00:00,2020-09-30 17:51:50.206000+00:00,14
4,goldfish@active.fi,active.fi,ThresholdCondition,190,姉,older sister,True,leitner,NaN,grid,1900,False,2020-09-30 17:51:51.245000+00:00,2020-09-30 17:51:53.477000+00:00,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85282,aster@active.fi,active.fi,ForwardCondition,1965,肖,resemblance,True,forward,exp_decay,grid,2639,False,2020-10-06 12:33:07.610000+00:00,2020-10-06 12:33:11.043000+00:00,12
85283,aster@active.fi,active.fi,ForwardCondition,1429,斉,simultaneous,True,forward,exp_decay,grid,2639,False,2020-10-06 12:33:17.376000+00:00,2020-10-06 12:33:19.775000+00:00,12
85284,aster@active.fi,active.fi,ForwardCondition,458,橋,bridge,True,forward,exp_decay,grid,2639,False,2020-10-06 12:33:22.841000+00:00,2020-10-06 12:33:26.554000+00:00,12
85285,aster@active.fi,active.fi,ForwardCondition,398,期,period of time,True,forward,exp_decay,grid,2639,False,2020-10-06 12:33:32.375000+00:00,2020-10-06 12:33:34.619000+00:00,12


In [ ]:
# Keep only users from the last experiment and that did it until the end (6 + 1  sessions for each teacher)
df.drop(df[(df.domain != "active.fi") | (df.n_session_done != 14)].index,
        inplace=True)

# Convert timestamps in datetime 
df["ts_display"] = pd.to_datetime(df["ts_display"]) 
df["ts_reply"] = pd.to_datetime(df["ts_reply"])

# Convert timestamps into seconds
beginning_history = pd.Timestamp("1970-01-01", tz="UTC")
df["timestamp"] = (df["ts_reply"] - beginning_history).dt.total_seconds().values

# Copy actual item ID in a new column
df["item_id"] = df.item
# Create new ids starting from zero
for i, i_id in enumerate(df.item_id.unique()):
    df.loc[df.item_id == i_id, 'item'] = i

# Total number of user
n_u = len(df.user.unique())

# Number of observations per user
n_o_by_u = np.zeros(shape=n_u, dtype=int)
for u, (user, user_df) in enumerate(df.groupby("user")):
    # Do not count first presentation
    n_o_by_u[u] = len(user_df) - len(user_df.item.unique()) 

# Total number of observation
n_obs = n_o_by_u.sum()

# Replies (1: success, 0: error)
y = np.zeros(shape=n_obs, dtype=int)
# Time elapsed since the last presentation
x = np.zeros(shape=n_obs, dtype=float)
# Number of repetition (number of presentation - 1)
r = np.zeros(shape=n_obs, dtype=int)
# Item ID
w = np.zeros(shape=n_obs, dtype=int)
# User ID
u = np.zeros(shape=n_obs, dtype=int)

# 
idx = 0
for i_u, (user, user_df) in enumerate(df.groupby("user")):

    user_df = user_df.sort_values(by="timestamp")
    seen = user_df.item.unique()
    w_u = user_df.item.values  # Words
    ts_u = user_df.timestamp.values
    counts = {word: -1 for word in seen}
    last_pres = {word: None for word in seen}
    r_u = np.zeros(len(user_df))  # Number of repetitions
    x_u = np.zeros(r_u.shape)  # Time elapsed since last repetition
    for i, word in enumerate(w_u):
        ts = ts_u[i]
        r_u[i] = counts[word]
        if last_pres[word] is not None:
            x_u[i] = ts - last_pres[word]
        counts[word] += 1
        last_pres[word] = ts

    to_keep = r_u >= 0
    y_u = user_df.success.values[to_keep]
    r_u = r_u[to_keep]
    w_u = w_u[to_keep]
    x_u = x_u[to_keep]

    n_ou = len(y_u)
    # assert n_o_by_u[i_u] == n_ou

    y[idx:idx + n_ou] = y_u
    x[idx:idx + n_ou] = x_u
    w[idx:idx + n_ou] = w_u
    r[idx:idx + n_ou] = r_u
    u[idx:idx + n_ou] = i_u

    idx += n_ou

data = {
    'u': u, 'w': w,
    'x': torch.from_numpy(x.reshape(-1, 1)),
    'r': torch.from_numpy(r.astype(float).reshape(-1, 1)),
    'y': torch.from_numpy(y.astype(float).reshape(-1, 1))
}

n_w = len(np.unique(w))
n_o_max = n_o_by_u.max()
n_o_min = n_o_by_u.min()
print("number of user", n_u)
print("number of items", n_w)
print("total number of observations (excluding first presentation)", n_obs)
print("minimum number of observation for a single user", n_o_min)
print("maximum number of observation for a single user", n_o_max)